<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Summary_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this notebook we will load the summarier models and generate summary we will save them and later evaluate in another notebook

In [ ]:
!git clone https://github.com/dmmiller612/lecture-summarizer.git
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)
!pip install bert-extractive-summarizer

In [13]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm
import os
import pandas as pd

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
all_files = os.listdir("/content/data/")   
contents = []
for file_name in tqdm(all_files):
  dir_name = "/content/data/" + file_name
  with open(dir_name) as f:
    content = f.read()
    contents.append(content)

df = pd.DataFrame()

df['content'] = contents
df.head()

100%|██████████| 13/13 [00:00<00:00, 6690.30it/s]


,content
0,The most sophisticated form of HIE creates sem...
1,Welcome back to pattern recognition.\nToday we...
2,Welcome back to Pattern Recognition.\nToday we...
3,Welcome back to Pattern Recognition!\nToday we...
4,Welcome back to Pattern Recognition!\nToday we...


In [24]:
import textwrap

def generate_summary(pipe, paragraph):
  arr = textwrap.wrap(paragraph, 4000)
  summary = ''
  for i in arr:
    summary += pipe( i ) [0]["summary_text"].replace(" .<n>", ".\n")
  return summary

## BART

In [ ]:
from transformers import pipeline, set_seed

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
bart_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(pipe, df['content'][i] ) 
  bart_summaries.append( summary )

df['bart'] = bart_summaries

100%|██████████| 4/4 [02:34<00:00, 38.66s/it]


In [ ]:
df.head()

,name,content,bart
0,news,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ..."
1,allhealth,Dr. Karen Bell is the chair of the Certificat...,Dr. Karen Bell is the chair of the Certificati...
2,health_today,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...
3,rfl_1,"All right, so here is a rule we're going to c...","The TD (0) rule looks like this, not so unfami..."


## PEGASUS

In [ ]:
#hide_output
from transformers import pipeline, set_seed
#hide_output

pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
pegasus_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(pipe, df['content'][i] ) 
  pegasus_summaries.append( summary )

df['pegasus'] = pegasus_summaries
df.head()

100%|██████████| 4/4 [05:57<00:00, 89.28s/it]


,name,content,bart,pegasus
0,news,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...
1,allhealth,Dr. Karen Bell is the chair of the Certificat...,Dr. Karen Bell is the chair of the Certificati...,The promise of Accountable Care is tempered by...
2,health_today,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...
3,rfl_1,"All right, so here is a rule we're going to c...","The TD (0) rule looks like this, not so unfami...",The TD (0) rule is used to calculate the maxim...


## BERT Extractive 

In [ ]:
from summarizer import Summarizer

summarizer = Summarizer()

bert_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = summarizer(df['content'][i] ) 
  bert_summaries.append( summary )

df['bert_extractive'] = bert_summaries
df.head()

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

100%|██████████| 4/4 [00:12<00:00,  3.07s/it]


,name,content,bart,pegasus,bert_extractive
0,news,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,"The Chrysler Building, the famous art deco New..."
1,allhealth,Dr. Karen Bell is the chair of the Certificat...,Dr. Karen Bell is the chair of the Certificati...,The promise of Accountable Care is tempered by...,Dr. Karen Bell is the chair of the Certificati...
2,health_today,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...,The most sophisticated form of HIE creates sem...
3,rfl_1,"All right, so here is a rule we're going to c...","The TD (0) rule looks like this, not so unfami...",The TD (0) rule is used to calculate the maxim...,"All right, so here is a rule we're going to ca..."


## DistilBert Base Uncased

In [ ]:
summarizer = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True) 

dist_bert_summaries = [] 
for i in tqdm(range( len(df) ) ): 
  summary = summarizer(df['content'][i] ) 
  dist_bert_summaries.append( summary ) 

df['dist_bert_extractive'] = dist_bert_summaries 

df.head() 

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

100%|██████████| 4/4 [00:03<00:00,  1.10it/s]


,name,content,bart,pegasus,bert_extractive,dist_bert_extractive
0,news,"The Chrysler Building, the famous art deco New...","The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,"The Chrysler Building, the famous art deco New...","The Chrysler Building, the famous art deco New..."
1,allhealth,Dr. Karen Bell is the chair of the Certificat...,Dr. Karen Bell is the chair of the Certificati...,The promise of Accountable Care is tempered by...,Dr. Karen Bell is the chair of the Certificati...,Dr. Karen Bell is the chair of the Certificati...
2,health_today,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...,The most sophisticated form of HIE creates sem...,The most sophisticated form of HIE creates sem...
3,rfl_1,"All right, so here is a rule we're going to c...","The TD (0) rule looks like this, not so unfami...",The TD (0) rule is used to calculate the maxim...,"All right, so here is a rule we're going to ca...","All right, so here is a rule we're going to ca..."


In [ ]:
%cd ..
df.to_csv('summaries.csv')

/content


In [ ]:
len(df.loc[2]['content'])

3990

In [ ]:
summarizer(df.loc[2]['content'][100:1000])

'the same concepts can be expressed by providers and represented in EHRs and other clinical systems. Here on the right, you see that this governance enables a number of value-added services.'

In [ ]:
df.loc[2]['content']